# <p style="text-align: center;, font-style: strong;">Partie 2.2 : MNIST with Multiple Layer Perceptron (MLP)</p>


## Dependencies

In [1]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("darwin-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [2]:
import org.platanios.tensorflow.api.ops.{Files, Image => TImage}
import java.nio.file.{ Files => JFiles, Paths, Path }

import java.awt.{Image => JImage}
import java.io.{BufferedInputStream, File, FileInputStream, ByteArrayOutputStream, IOException}
import scala.io.Source
import scala.collection.mutable.ListBuffer
import java.awt.image.BufferedImage
import java.awt.Color
import java.awt.image.BufferedImage
import javax.imageio.ImageIO
import scala.util.Random
import java.util.Base64
import java.io.PrintWriter
import scala.io.Source
import java.io.ByteArrayInputStream
import org.platanios.tensorflow.api.tf
import org.platanios.tensorflow.api.tensors.Tensor
import org.platanios.tensorflow.api.core.Shape
import org.platanios.tensorflow.api.core.client.{Session, FeedMap}
import org.platanios.tensorflow.api.core.types.DataType
import org.platanios.tensorflow.data.image.MNISTLoader
import java.nio.file.{Files, Paths}
import org.platanios.tensorflow.api.core.{Shape, types}
import scala.language.postfixOps
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.core.types.UByte
import org.platanios.tensorflow.api.implicits.helpers.{OutputStructure, OutputToDataType, OutputToShape}
import org.platanios.tensorflow.api.learn.ClipGradientsByGlobalNorm
import org.platanios.tensorflow.api.ops.Output
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.learn.layers.{Flatten, Input, Linear, ReLU, SparseSoftmaxCrossEntropy, Mean, ScalarSummary}
import org.platanios.tensorflow.api.learn.{Model, StopCriteria, Configuration}
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.api.learn.hooks.{SummarySaver, StepHookTrigger, CheckpointSaver}
import org.platanios.tensorflow.api.config.TensorBoardConfig
import java.util.Base64
import org.tensorflow.framework.GraphDef



import org.platanios.tensorflow.api.ops.{Files, Image => TImage}

import java.nio.file.{ Files => JFiles, Paths, Path }


import java.awt.{Image => JImage}

import java.io.{BufferedInputStream, File, FileInputStream, ByteArrayOutputStream, IOException}

import scala.io.Source

import scala.collection.mutable.ListBuffer

import java.awt.image.BufferedImage

import java.awt.Color

import java.awt.image.BufferedImage

import javax.imageio.ImageIO

import scala.util.Random

import java.util.Base64

import java.io.PrintWriter

import scala.io.Source

import java.io.ByteArrayInputStream

import org.platanios.tensorflow.api.tf

import org.platanios.tensorflow.api.tensors.Tensor

import org.platanios.tensorflow.api.core.Shape

import org.platanios.tensorflow.api.core.client.{Session, FeedMap}

import org.platanios.tensorflow.api.core.types.DataType

import org.platanios.tensorflow.data.image.MNISTLoader

import java.nio.file.{Files, Paths}

import org.platanios.tensorflow.api.core.{Shape, type

## Display MNIST Dataset

In [2]:
lazy val graph = Graph.fromGraphDef(GraphDef.parseFrom(new BufferedInputStream(new FileInputStream(new File("../resources/save/graph.pbtxt")))))
val session = Session(graph)
session.run(fetches = , feeds = )

(console):3:23 expected (If | While | Try | DoWhile | For | Throw | Return | ImplicitLambda | SmallerExprOrLambda)
session.run(fetches = , feeds = )
                      ^

: 

In [ ]:
val imagePlaceholder = graph.getOutputByName("OutputLayer/Linear:0").toUByte



In [ ]:
def accuracy(images: Tensor[UByte], labels: Tensor[UByte]): Float = {
    val predictions = estimator.infer(() => images.toFloat)
    predictions
      .argmax(1).toUByte
      .equal(labels).toFloat
      .mean().scalar
}

println(s"Train accuracy = ${accuracy(dataSet.trainImages, dataSet.trainLabels)}")
println(s"Test accuracy = ${accuracy(dataSet.testImages, dataSet.testLabels)}")

In [ ]:
def transformImagesForPrediction(imagefilePath: String, numChannel: Int, s: Session = Session()): Tensor[UByte] = {
    val tensorImageOuts = TImage.decodePng(Files.readFile(imagefilePath), numChannel)
    s.run(fetches = tf.reshape(tensorImageOuts, Shape(1,28,28)))
}

def runPrediction(imagePath: String): String = {
    val imageTest = transformImagesForPrediction(imagePath, 1)
    val imageToDisplay = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(tf.reshape(imageTest, Shape(28, 28, 1)))))
    Image(imageToDisplay.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    val predictions = estimator.infer(() => imageTest.toFloat)
    s"Le chiffre est : ${predictions.argmax(1).toInt.scalar}"
}

//runPrediction("../resources/images/de677a85-be92-4f72-8a9a-f97d18be26e5.png")



In [ ]:
def drawOwnNumber(id: String) {
    val htmlCode = JFiles.readString(Paths.get("../resources/drawing.html"))
    kernel.publish.display(almond.interpreter.api.DisplayData(Map("text/html" -> htmlCode.replace("nameCommandToChange", id))))
}
val id = java.util.UUID.randomUUID().toString // random ID use to synchronize communication between JS code and Scala code
drawOwnNumber(id)
kernel.publish.html("", id)
kernel.comm.receiver(id) { data =>
    kernel.publish.updateHtml("Waiting", id)
    val nameImage = java.util.UUID.randomUUID().toString
    val base64 = data.drop(34).dropRight(2)
    val bis = new ByteArrayInputStream(Base64.getDecoder().decode(base64))
    val bImage2 = ImageIO.read(bis)
    val tmp = bImage2.getScaledInstance(28, 28, JImage.SCALE_SMOOTH)
    val dimg = new BufferedImage(28, 28, BufferedImage.TYPE_INT_ARGB)
    val g2d = dimg.createGraphics()
    g2d.drawImage(tmp, 0, 0, null)
    g2d.dispose()
    ImageIO.write(dimg, "png", new File(s"../resources/images/$nameImage.png") )
    //val result = runPrediction(s"../resources/images/$nameImage.png")
    //kernel.publish.updateHtml(s"$result", id)
}